### Mounting Googel Drive

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Importing Dependencies

In [2]:
import os
!pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html

import numpy as np
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import urllib.request
import pandas as pd
import dgl.data
import dgl
from dgl.data import DGLDataset
import torch
import os
import itertools
import dgl.nn as dglnn
from dgl.nn import GraphConv
from dgl.nn import GATConv


from scipy.spatial import Delaunay
from sklearn.metrics import f1_score

Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 165.0 MB 36 kB/s 


DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


### Reding CSV files defining the classes, edges and node feaures respectively. More details can be found at: https://www.kaggle.com/ellipticco/elliptic-data-set


NOTE: Please change the path of the CSV files according to your directory structure.

In [3]:
classes = pd.read_csv('/content/gdrive/MyDrive/Fall_21/BC_DL/elliptic_bitcoin_dataset/elliptic_txs_classes.csv')
edges = pd.read_csv('/content/gdrive/MyDrive/Fall_21/BC_DL/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')
features = pd.read_csv('/content/gdrive/MyDrive/Fall_21/BC_DL/elliptic_bitcoin_dataset/elliptic_txs_features.csv',header=None).set_index(0,verify_integrity=True)

### Filtering entries with unknown classes.

In [4]:
classes_filtered = classes
classes_filtered = classes_filtered[classes_filtered['class'] != 'unknown']

### Spliting features into 2 sections: i) all entries with 1st feature value below 35 would be used for training ii) all entries with 2nd feature value above 35 would be used for testing.

In [5]:
features_train = features[features[1]<35]
features_test = features[features[1]>=35]

### Creating Training & testing dataset

In [6]:
src = []
dst = []
feats = []
labels = []

id_cnt = 0

id_dict = {}

for index, row in edges.iterrows():
  
  if (len(classes_filtered.loc[classes_filtered['txId']==row['txId1']]['class'].values) != 0) and (len(classes_filtered.loc[classes_filtered['txId']==row['txId2']]['class'].values) != 0) and (row['txId1'] in features_train.index) and (row['txId2'] in features_train.index):
    
    key = row['txId1']
    if key not in id_dict:
        id_dict.update({key: id_cnt})
        src.append(id_cnt)
        feats.append(features.loc[key].to_numpy())

        #print("labe",int(classes_filtered.loc[classes_filtered['txId']==row['txId1']]['class'].values[0]))
        labels.append(int(classes_filtered.loc[classes_filtered['txId']==row['txId1']]['class'].values[0])-1)
       

        
        id_cnt += 1
    else:
        val = id_dict[key]
        src.append(val)
      

    key = row['txId2']
    if key not in id_dict:
        id_dict.update({key: id_cnt})
        dst.append(id_cnt)
        feats.append(features.loc[key].to_numpy())
        
        #print("labe",int(classes_filtered.loc[classes_filtered['txId']==row['txId2']]['class'].values[0]))
        labels.append(int(classes_filtered.loc[classes_filtered['txId']==row['txId2']]['class'].values[0])-1)

        id_cnt += 1
    else:
        val = id_dict[key]
        dst.append(val)
       

In [7]:
src_test = []
dst_test = []
feats_test = []
labels_test = []

id_cnt = 0

id_dict_test = {}

for index, row in edges.iterrows():
  
  if (len(classes_filtered.loc[classes_filtered['txId']==row['txId1']]['class'].values) != 0) and (len(classes_filtered.loc[classes_filtered['txId']==row['txId2']]['class'].values) != 0) and (row['txId1'] in features_test.index) and (row['txId2'] in features_test.index):
    
    key = row['txId1']
    if key not in id_dict_test:
        id_dict_test.update({key: id_cnt})
        src_test.append(id_cnt)
        feats_test.append(features.loc[key].to_numpy())
        labels_test.append(int(classes_filtered.loc[classes_filtered['txId']==row['txId1']]['class'].values[0])-1)
        id_cnt += 1
    else:
        val = id_dict_test[key]
        src_test.append(val)
      

    key = row['txId2']
    if key not in id_dict_test:
        id_dict_test.update({key: id_cnt})
        dst_test.append(id_cnt)
        feats_test.append(features.loc[key].to_numpy())
        labels_test.append(int(classes_filtered.loc[classes_filtered['txId']==row['txId2']]['class'].values[0])-1)
        id_cnt += 1
    else:
        val = id_dict_test[key]
        dst_test.append(val)
       
     



### Creating Training & Testing graphs using DGL APIs

In [8]:
# Training Graph
feats = np.array(feats)
g = dgl.graph((src, dst))
g.ndata['feat'] = torch.from_numpy(feats)
g.ndata['label'] = torch.from_numpy(np.array(labels))

In [9]:
# Testing Graph
feats_test = np.array(feats_test)
g_test = dgl.graph((src_test, dst_test))
g_test.ndata['feat'] = torch.from_numpy(feats_test)
g_test.ndata['label'] = torch.from_numpy(np.array(labels_test))

### Defining GCN using DGL API

In [10]:
class AML_GCN(nn.Module):
    def __init__(self, input_feats, hidden_feats, output_feats):
        super().__init__()
        self.layer1 = dglnn.SAGEConv(
            in_feats=input_feats, out_feats=hidden_feats, aggregator_type='mean')
        self.layer2 = dglnn.SAGEConv(
            in_feats=hidden_feats, out_feats=output_feats, aggregator_type='mean')

    def forward(self, graph, inputs):
        h = self.layer1(graph, inputs)
        h = F.relu(h)
        h = self.layer2(graph, h)
        return h

### Training and evaluating GCN

In [12]:
node_features = g.ndata['feat'].float()
node_labels = g.ndata['label'].long()
node_features_test = g_test.ndata['feat'].float()
node_labels_test = g_test.ndata['label'].long()

n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

def evaluate(model, graph, features, labels, final=0):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits
        labels = labels
        e, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        
        #f1_m = f1_score(indices.cpu().numpy(), labels.cpu().numpy(), average='micro')
        f1 = f1_score(indices.cpu().numpy(), labels.cpu().numpy(), average=None)
        #f1_w = f1_score(indices.cpu().numpy(), labels.cpu().numpy(), average='weighted')

        print("Epoch F1 score:",(f1[0]+f1[1])/2)
        print("Epoch MicroAvg F1 score:",f1[1])

        if final==1:
          print("Final F1 score:",(f1[0]+f1[1])/2)
          print("Final MicroAvg F1 score:",f1[1])


        return correct.item() * 1.0 / len(labels)


model = AML_GCN(input_feats=n_features, hidden_feats=100, output_feats=n_labels)
opt = torch.optim.Adam(model.parameters(),lr=0.001)

for epoch in range(1000):
    model.train()
    
    logits = model(g, node_features)
    
    loss = F.cross_entropy(logits, node_labels, weight=torch.tensor([0.7,0.03]))
   
    if (epoch+1)%50==0:
      acc = evaluate(model, g_test, node_features_test, node_labels_test)
    
    opt.zero_grad()
    loss.backward()
    opt.step()

acc = evaluate(model, g_test, node_features_test, node_labels_test, 1)

Epoch F1 score: 0.5090068470228363
Epoch MicroAvg F1 score: 0.8181008020596099
Epoch F1 score: 0.5310026266357524
Epoch MicroAvg F1 score: 0.8479684190256114
Epoch F1 score: 0.5506878005508433
Epoch MicroAvg F1 score: 0.8713868750588456
Epoch F1 score: 0.5717969719056701
Epoch MicroAvg F1 score: 0.8909023668639052
Epoch F1 score: 0.5840364812762884
Epoch MicroAvg F1 score: 0.9022762592558735
Epoch F1 score: 0.5950968056653602
Epoch MicroAvg F1 score: 0.9121266968325791
Epoch F1 score: 0.6080208075023104
Epoch MicroAvg F1 score: 0.9207503141267277
Epoch F1 score: 0.6189932140957974
Epoch MicroAvg F1 score: 0.927364337628751
Epoch F1 score: 0.626008417342968
Epoch MicroAvg F1 score: 0.9316284851713728
Epoch F1 score: 0.6326382921277028
Epoch MicroAvg F1 score: 0.9356264921743744
Epoch F1 score: 0.6361409452359381
Epoch MicroAvg F1 score: 0.9380015874415732
Epoch F1 score: 0.6405162799422318
Epoch MicroAvg F1 score: 0.9406902615959551
Epoch F1 score: 0.6453055971755352
Epoch MicroAvg F1 s

### Training a new "custom GCN" which use an "Attention Layer".

In [13]:
class GAT_GCN(nn.Module):
    def __init__(self, input_feats, hidden_feats, output_feats, allow_zero_in_degree = True):
        super().__init__()
        self.conv1 = dglnn.SAGEConv(
            in_feats=input_feats, out_feats=hidden_feats, aggregator_type='mean')
        self.conv2 = dglnn.SAGEConv(
            in_feats=hidden_feats, out_feats=output_feats, aggregator_type='mean')
        
        self.layer1 = GATConv(hidden_feats, hidden_feats, 1,allow_zero_in_degree = True)
        

    def forward(self, graph, inputs):
        
        h = self.conv1(graph, inputs)
        h = F.relu(h)
        h = self.layer1(graph,h)
        h = torch.squeeze(h,1)
        h = self.conv2(graph, h)
        return h


node_features = g.ndata['feat'].float()
node_labels = g.ndata['label'].long()
node_features_test = g_test.ndata['feat'].float()
node_labels_test = g_test.ndata['label'].long()

n_features = node_features.shape[1]
n_labels = int(node_labels.max().item() + 1)

def evaluate(model, graph, features, labels, final=0):
    model.eval()
    with torch.no_grad():
        logits = model(graph, features)
        logits = logits
        labels = labels
        e, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        
        #f1_m = f1_score(indices.cpu().numpy(), labels.cpu().numpy(), average='micro')
        f1 = f1_score(indices.cpu().numpy(), labels.cpu().numpy(), average=None)
        #f1_w = f1_score(indices.cpu().numpy(), labels.cpu().numpy(), average='weighted')

        print("Epoch F1 score:",(f1[0]+f1[1])/2)
        print("Epoch MicroAvg F1 score:",f1[1])

        if final==1:
          print("Final F1 score:",(f1[0]+f1[1])/2)
          print("Final MicroAvg F1 score:",f1[1])


        return correct.item() * 1.0 / len(labels)


model = GAT_GCN(input_feats=n_features, hidden_feats=100, output_feats=n_labels)
opt = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    g = dgl.add_self_loop(g)
    model.train()
    
    logits = model(g, node_features)
    
    loss = F.cross_entropy(logits, node_labels, weight=torch.tensor([0.7,0.03]))
    
    if (epoch+1)%50==0:
      acc = evaluate(model, g_test, node_features_test, node_labels_test)
  
    opt.zero_grad()
    loss.backward()
    opt.step()


acc = evaluate(model, g_test, node_features_test, node_labels_test, 1)

Epoch F1 score: 0.2743930414082318
Epoch MicroAvg F1 score: 0.43900835141711053
Epoch F1 score: 0.30061744601665563
Epoch MicroAvg F1 score: 0.4895833333333333
Epoch F1 score: 0.3170200670317881
Epoch MicroAvg F1 score: 0.5190212315546977
Epoch F1 score: 0.4999777519126003
Epoch MicroAvg F1 score: 0.8849484912024796
Epoch F1 score: 0.5053887296078332
Epoch MicroAvg F1 score: 0.8922589406971478
Epoch F1 score: 0.5108129955376439
Epoch MicroAvg F1 score: 0.8993210126354603
Epoch F1 score: 0.5145598966168178
Epoch MicroAvg F1 score: 0.9044785468211713
Epoch F1 score: 0.521179691519089
Epoch MicroAvg F1 score: 0.9111120992485214
Epoch F1 score: 0.5237106828921976
Epoch MicroAvg F1 score: 0.9148097284366279
Epoch F1 score: 0.5270564097329438
Epoch MicroAvg F1 score: 0.9173922651933702
Epoch F1 score: 0.528596922356796
Epoch MicroAvg F1 score: 0.9197530864197531
Epoch F1 score: 0.5294919668757396
Epoch MicroAvg F1 score: 0.9208544373486016
Epoch F1 score: 0.5310853215553821
Epoch MicroAvg F1